<a href="https://colab.research.google.com/github/denizgul/token-classification/blob/master/token_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install seqeval

     |████████████████████████████████| 51kB 4.7MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=2e12f88a6048b3005caa60732909ed012521dda0b3d561351741daa9f0c2df90
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [2]:
!pip install transformers

     |████████████████████████████████| 1.5MB 11.1MB/s 
     |████████████████████████████████| 890kB 43.9MB/s 
     |████████████████████████████████| 2.9MB 50.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=d717bb6a0c6cf5f4d22ec5619852a439375dbea4a94be7eaedd701b1fd60af81
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import tensorflow as tf
import torch

# check GPU
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    device = torch.device("cuda")
    print('GPU:', torch.cuda.get_device_name(0))
else:
    raise SystemError('GPU device not found')

SystemError: ignored

In [4]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
!ls gdrive/MyDrive/token-classification/input-thesis

cached_dev_BertTokenizer_128	     dev-original.txt
cached_dev_BertTokenizer_128.lock    dev.txt
cached_test_BertTokenizer_128	     extra
cached_test_BertTokenizer_128.lock   labels.txt
cached_train_BertTokenizer_128	     test.txt
cached_train_BertTokenizer_128.lock  train.txt


In [7]:
import sys
sys.path.append('/content/gdrive/MyDrive/token-classification')

In [8]:
!pip install easydict

In [9]:
import json 
import csv 
from google.colab import files
from easydict import EasyDict as edict
#upload a config file
uploaded = files.upload()
print(uploaded)


Saving config.json to config.json
{'config.json': b'{\n    "data_dir": "./gdrive/MyDrive/token-classification/input-thesis",\n    "labels": "./gdrive/MyDrive/token-classification/input-thesis/labels.txt",\n    "model_name_or_path": "dbmdz/bert-base-turkish-cased",\n    "output_dir": "../output/BERT-WIKIANN",\n    "max_seq_length": 128,\n    "num_train_epochs": 1,\n    "per_device_train_batch_size": 16,\n    "save_steps": 750,\n    "seed": 1,\n    "do_train": true,\n    "do_eval": true,\n    "do_predict": true,\n    "overwrite_output_dir": true\n}'}


In [ ]:
#!rm -rf *.json

In [10]:
!ls

config.json  gdrive  sample_data


In [11]:

data = next(iter(uploaded.values()))
json.loads(data.decode())

args = edict(json.loads(data.decode()))

In [13]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Fine-tuning the library models for named entity recognition on CoNLL-2003. """
import logging
import os
import sys
from dataclasses import dataclass, field
from importlib import import_module
from typing import Dict, List, Optional, Tuple

import numpy as np
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch import nn

from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)
from utils_ner import Split, TokenClassificationDataset, TokenClassificationTask


logger = logging.getLogger(__name__)

In [15]:

@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    task_type: Optional[str] = field(
        default="NER", metadata={"help": "Task type to fine tune in training (e.g. NER, POS, etc)"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    use_fast: bool = field(default=False, metadata={"help": "Set this flag to use fast tokenization."})
    # If you want to tweak more attributes on your tokenizer, you should do it in a distinct script,
    # or just modify its tokenizer_config.json.
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )

In [16]:

@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    data_dir: str = field(
        metadata={"help": "The input data dir. Should contain the .txt files for a CoNLL-2003-formatted task."}
    )
    labels: Optional[str] = field(
        default=None,
        metadata={"help": "Path to a file containing all labels. If not specified, CoNLL-2003 labels are used."},
    )
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )



In [17]:
# See all possible arguments in src/transformers/training_args.py
# or by passing the --help flag to this script.
# We now keep distinct sets of args, for a cleaner separation of concerns.

parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
if uploaded:
    model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath("config.json"))

model_args.model_name_or_path

'dbmdz/bert-base-turkish-cased'

In [ ]:
if (
    os.path.exists(training_args.output_dir)
    and os.listdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):
    raise ValueError(
        f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
    )

module = import_module("tasks")
try:
    token_classification_task_clazz = getattr(module, model_args.task_type)
    token_classification_task: TokenClassificationTask = token_classification_task_clazz()
except AttributeError:
    raise ValueError(
        f"Task {model_args.task_type} needs to be defined as a TokenClassificationTask subclass in {module}. "
        f"Available tasks classes are: {TokenClassificationTask.__subclasses__()}"
    )

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
)
logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    training_args.local_rank,
    training_args.device,
    training_args.n_gpu,
    bool(training_args.local_rank != -1),
    training_args.fp16,
)
logger.info("Training/evaluation parameters %s", training_args)

# Set seed
set_seed(training_args.seed)

# Prepare CONLL-2003 task
labels = token_classification_task.get_labels(data_args.labels)
label_map: Dict[int, str] = {i: label for i, label in enumerate(labels)}
num_labels = len(labels)

# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    num_labels=num_labels,
    id2label=label_map,
    label2id={label: i for i, label in enumerate(labels)},
    cache_dir=model_args.cache_dir,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast,
)
model = AutoModelForTokenClassification.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
)

# Get datasets
train_dataset = (
    TokenClassificationDataset(
        token_classification_task=token_classification_task,
        data_dir=data_args.data_dir,
        tokenizer=tokenizer,
        labels=labels,
        model_type=config.model_type,
        max_seq_length=data_args.max_seq_length,
        overwrite_cache=data_args.overwrite_cache,
        mode=Split.train,
    )
    if training_args.do_train
    else None
)
eval_dataset = (
    TokenClassificationDataset(
        token_classification_task=token_classification_task,
        data_dir=data_args.data_dir,
        tokenizer=tokenizer,
        labels=labels,
        model_type=config.model_type,
        max_seq_length=data_args.max_seq_length,
        overwrite_cache=data_args.overwrite_cache,
        mode=Split.dev,
    )
    if training_args.do_eval
    else None
)

def align_predictions(predictions: np.ndarray, label_ids: np.ndarray) -> Tuple[List[int], List[int]]:
    preds = np.argmax(predictions, axis=2)

    batch_size, seq_len = preds.shape

    out_label_list = [[] for _ in range(batch_size)]
    preds_list = [[] for _ in range(batch_size)]

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != nn.CrossEntropyLoss().ignore_index:
                out_label_list[i].append(label_map[label_ids[i][j]])
                preds_list[i].append(label_map[preds[i][j]])

    return preds_list, out_label_list

def compute_metrics(p: EvalPrediction) -> Dict:
    preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)
    return {
        "accuracy_score": accuracy_score(out_label_list, preds_list),
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Training
if training_args.do_train:
    trainer.train(
        model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
    )
    trainer.save_model()
    # For convenience, we also re-save the tokenizer to the same directory,
    # so that you can share your model easily on huggingface.co/models =)
  #  if trainer.is_world_master():
  #      tokenizer.save_pretrained(training_args.output_dir)

# Evaluation
results = {}
if training_args.do_eval:
    logger.info("*** Evaluate ***")

    result = trainer.evaluate()

    output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
    if trainer.is_world_process_zero():
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key, value in result.items():
                logger.info("  %s = %s", key, value)
                writer.write("%s = %s\n" % (key, value))

        results.update(result)

# Predict
if training_args.do_predict:
    test_dataset = TokenClassificationDataset(
        token_classification_task=token_classification_task,
        data_dir=data_args.data_dir,
        tokenizer=tokenizer,
        labels=labels,
        model_type=config.model_type,
        max_seq_length=data_args.max_seq_length,
        overwrite_cache=data_args.overwrite_cache,
        mode=Split.test,
    )

    predictions, label_ids, metrics = trainer.predict(test_dataset)
    preds_list, _ = align_predictions(predictions, label_ids)

    output_test_results_file = os.path.join(training_args.output_dir, "test_results.txt")
    if trainer.is_world_process_zero():
        with open(output_test_results_file, "w") as writer:
            for key, value in metrics.items():
                logger.info("  %s = %s", key, value)
                writer.write("%s = %s\n" % (key, value))

    # Save predictions
    output_test_predictions_file = os.path.join(training_args.output_dir, "test_predictions.txt")
    if trainer.is_world_process_zero():
        with open(output_test_predictions_file, "w") as writer:
            with open(os.path.join(data_args.data_dir, "input-thesis/test.txt"), "r") as f:
                token_classification_task.write_predictions_to_file(writer, f, preds_list)

return results



01/06/2021 00:12:13 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
01/06/2021 00:12:13 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../output/BERT-WIKIANN', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, model_parallel=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1, max_steps=-1, warmup_steps=0, logging_dir='runs/Jan06_00-12-07_1014ec7ab74d', logging_first_step=False, logging_steps=500, save_steps=750, save_total_limit=None, no_cuda=False, seed=1, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_co

01/06/2021 00:12:15 - INFO - filelock -   Lock 139642275871768 released on /root/.cache/huggingface/transformers/4bcf91c5f5c2b69c278c1fa439966e8f119b238f647a4e7e5b8ac163ecab8f8d.d34b61340faaa3122b620c12ba54ab8b1016bfeb2ff97c1d5e80b86808783b7a.lock


01/06/2021 00:12:15 - INFO - filelock -   Lock 139642276471248 acquired on /root/.cache/huggingface/transformers/f28b70f1322047260701edc1b83924cb958afb70581d9a8e95644cc551a00431.2439760968a7b05b87c85c85cd02e8aba6e01e526263c6be3a99b2d64c4cee1d.lock


01/06/2021 00:12:16 - INFO - filelock -   Lock 139642276471248 released on /root/.cache/huggingface/transformers/f28b70f1322047260701edc1b83924cb958afb70581d9a8e95644cc551a00431.2439760968a7b05b87c85c85cd02e8aba6e01e526263c6be3a99b2d64c4cee1d.lock


01/06/2021 00:12:17 - INFO - filelock -   Lock 139642276471248 acquired on /root/.cache/huggingface/transformers/a615e3b5673a9da0c1cdc2758fecaabb1fb1df6c3e83a7b6c8503055c1da8d49.c70618325b9fc2d2d041e439766d360b48a086a8841cc2896322f6b8aefc0225.lock


01/06/2021 00:12:17 - INFO - filelock -   Lock 139642276471248 released on /root/.cache/huggingface/transformers/a615e3b5673a9da0c1cdc2758fecaabb1fb1df6c3e83a7b6c8503055c1da8d49.c70618325b9fc2d2d041e439766d360b48a086a8841cc2896322f6b8aefc0225.lock


01/06/2021 00:12:18 - INFO - filelock -   Lock 139642275871712 acquired on /root/.cache/huggingface/transformers/380745366fbadf49708b28994fcfc6582de8b8e8395dac38e107814aa41d26f4.283415ff1b308f25b2c470c13745106c3ad576c9505242ff7763e54902d997a4.lock


01/06/2021 00:12:34 - INFO - filelock -   Lock 139642275871712 released on /root/.cache/huggingface/transformers/380745366fbadf49708b28994fcfc6582de8b8e8395dac38e107814aa41d26f4.283415ff1b308f25b2c470c13745106c3ad576c9505242ff7763e54902d997a4.lock


Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initiali

Step,Training Loss
